In [1]:
import requests
import json
import pandas as pd
import numpy as np
import os

In [2]:
## Get Geolocations for federal states and districts
with open('C:/Users/simon/Jupyter/Covid/Dashboard/landkreise_simplify200.geojson', 'r') as read_file:
    geo_kreis = json.load(read_file)

with open('C:/Users/simon/Jupyter/Covid/Dashboard/bundeslaender_simplify200.geojson', 'r') as read_file:
    geo_land = json.load(read_file)

In [3]:
## Get population stats for federal states and districts
df_einwohner_kreis = pd.read_csv('C:/Users/simon/Jupyter/Covid/Dashboard/Einwohner_kreis.csv', encoding = 'ISO-8859-1', sep=';', dtype= {'RS': str, 'Kreis': str, 'Einwohner': int})

df_einwohner_land = pd.read_csv('C:/Users/simon/Jupyter/Covid//Dashboard/Einwohner_land.csv', encoding = 'ISO-8859-1', sep=';', dtype= {'RS': str, 'Kreis': str, 'Einwohner': int})
df_einwohner_land.head()

,Bundesland,Einwohner
0,Baden-Württemberg,11100394
1,Bayern,13124737
2,Berlin,3669491
3,Brandenburg,2521893
4,Bremen,681202


In [4]:
## Get Covid stats from RKI
covid_url = 'https://opendata.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.geojson'
r = requests.get(covid_url)
r.status_code

200

In [5]:
a = json.loads(r.content)
df = pd.json_normalize(a['features'])

df.head()

,type,geometry,properties.ObjectId,properties.IdBundesland,properties.Bundesland,properties.Landkreis,properties.Altersgruppe,properties.Geschlecht,properties.AnzahlFall,properties.AnzahlTodesfall,properties.Meldedatum,properties.IdLandkreis,properties.Datenstand,properties.NeuerFall,properties.NeuerTodesfall,properties.Refdatum,properties.NeuGenesen,properties.AnzahlGenesen,properties.IstErkrankungsbeginn,properties.Altersgruppe2
0,Feature,None,1,1,Schleswig-Holstein,SK Flensburg,A35-A59,M,2,0,2021-01-22T00:00:00Z,01001,"10.05.2021, 00:00 Uhr",0,-9,2021-01-18T00:00:00Z,0,2,1,Nicht übermittelt
1,Feature,None,2,1,Schleswig-Holstein,SK Flensburg,A35-A59,M,3,0,2021-01-22T00:00:00Z,01001,"10.05.2021, 00:00 Uhr",0,-9,2021-01-22T00:00:00Z,0,3,0,Nicht übermittelt
2,Feature,None,3,1,Schleswig-Holstein,SK Flensburg,A35-A59,M,1,0,2021-01-23T00:00:00Z,01001,"10.05.2021, 00:00 Uhr",0,-9,2021-01-23T00:00:00Z,0,1,0,Nicht übermittelt
3,Feature,None,4,1,Schleswig-Holstein,SK Flensburg,A35-A59,M,1,0,2021-01-24T00:00:00Z,01001,"10.05.2021, 00:00 Uhr",0,-9,2021-01-18T00:00:00Z,0,1,1,Nicht übermittelt
4,Feature,None,5,1,Schleswig-Holstein,SK Flensburg,A35-A59,M,1,0,2021-01-24T00:00:00Z,01001,"10.05.2021, 00:00 Uhr",0,-9,2021-01-24T00:00:00Z,0,1,0,Nicht übermittelt


In [6]:
## Clean-up and data preparation
df.columns = df.columns.str.replace('properties.', '')
df = df[['IdBundesland', 'Bundesland', 'Landkreis', 'IdLandkreis', 'AnzahlFall', 'Altersgruppe', 'AnzahlTodesfall', 'Geschlecht', 'Meldedatum']]
df['Meldedatum'] = df['Meldedatum'].str.replace('T00:00:00Z', '')
df['Meldedatum'] = pd.to_datetime(df['Meldedatum'])

df = df.merge(df_einwohner_kreis, how='left', left_on='IdLandkreis', right_on='RS')
df = df.merge(df_einwohner_land, how='left', left_on='Bundesland', right_on='Bundesland') 

df_gender = pd.get_dummies(df['Geschlecht']).mul(df['AnzahlFall'],0)
df_gender.columns = ['Sex_M', 'Sex_F', 'Sex_Unbekannt']
df = df.join(df_gender)

df_age = pd.get_dummies(df['Altersgruppe']).mul(df['AnzahlFall'],0)
df = df.join(df_age)



df.drop(['RS', 'Kreis'], axis=1, inplace=True)

df.head()

,IdBundesland,Bundesland,Landkreis,IdLandkreis,AnzahlFall,Altersgruppe,AnzahlTodesfall,Geschlecht,Meldedatum,Einwohner_x,...,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,A60-A79,A80+,unbekannt
0,1,Schleswig-Holstein,SK Flensburg,01001,2,A35-A59,0,M,2021-01-22,90164.0,...,2,0,0,0,0,0,2,0,0,0
1,1,Schleswig-Holstein,SK Flensburg,01001,3,A35-A59,0,M,2021-01-22,90164.0,...,3,0,0,0,0,0,3,0,0,0
2,1,Schleswig-Holstein,SK Flensburg,01001,1,A35-A59,0,M,2021-01-23,90164.0,...,1,0,0,0,0,0,1,0,0,0
3,1,Schleswig-Holstein,SK Flensburg,01001,1,A35-A59,0,M,2021-01-24,90164.0,...,1,0,0,0,0,0,1,0,0,0
4,1,Schleswig-Holstein,SK Flensburg,01001,1,A35-A59,0,M,2021-01-24,90164.0,...,1,0,0,0,0,0,1,0,0,0


In [7]:
df_bund = df.groupby(by=['Meldedatum']).agg({
    'AnzahlFall':'sum',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})


#df_bund.set_index('Region', append=True, inplace=True)
#df_bund = df_bund.swaplevel('Region', 'Meldedatum')

df_bund['Fall_Last7'] = df_bund['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_bund['Fall_MA7'] = df_bund['AnzahlFall'].rolling(7).mean()
df_bund['R7'] = (df_bund['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_bund['Einwohner'] = df_einwohner_land['Einwohner'].sum()
df_bund['Fall_100K'] = ((df_bund['Fall_Last7'] / df_bund['Einwohner']) * 100000).round(2)

df_bund.reset_index(inplace=True)
df_bund['Region'] = 'Deutschland'
df_bund['Typ'] = 'Nation'


df_bund.tail()

,Meldedatum,AnzahlFall,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,A60-A79,A80+,unbekannt,Fall_Last7,Fall_MA7,R7,Einwohner,Fall_100K,Region,Typ
453,2021-05-05,20917,21,10683,9993,241,801,2391,6270,8236,2650,543,26,113511.0,16215.857143,0.89,83166711,136.49,Deutschland,Nation
454,2021-05-06,17172,12,8632,8329,211,692,2021,5295,6480,2154,499,31,109203.0,15600.428571,0.87,83166711,131.31,Deutschland,Nation
455,2021-05-07,15136,3,7643,7326,167,632,1823,4708,5647,1896,414,16,105151.0,15021.571429,0.86,83166711,126.43,Deutschland,Nation
456,2021-05-08,11484,2,5653,5682,149,589,1477,3457,4256,1403,285,17,101561.0,14508.714286,0.86,83166711,122.12,Deutschland,Nation
457,2021-05-09,4020,2,2046,1913,61,190,451,1302,1494,462,113,8,98959.0,14137.000000,0.87,83166711,118.99,Deutschland,Nation


In [8]:
df_land = df.groupby(by=['Bundesland', 'Meldedatum']).agg({
    'IdBundesland': 'first',
    'AnzahlFall': 'sum',
    'Einwohner_y': 'first',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})

df_land.rename(columns={'Einwohner_y': 'Einwohner'}, inplace=True)

#df_land.index.names = ['Region', 'Meldedatum']

df_land['Fall_MA4'] = df_land.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(4).mean().values
df_land['Fall_MA7'] = df_land.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(7).mean().values

df_land['R4'] = df_land['Fall_MA4'].pct_change(periods=4) + 1
df_land['R7'] = (df_land['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_land['Fall_Last7'] = df_land['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_land['Fall_100K'] = ((df_land['Fall_Last7'] / df_land['Einwohner']) * 100000).round(2)

#

df_land.reset_index(inplace=True)
df_land['Region'] = df_land['Bundesland']
df_land['Typ'] = 'Bundesland'

df_land.tail()

,Bundesland,Meldedatum,IdBundesland,AnzahlFall,Einwohner,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,...,A80+,unbekannt,Fall_MA4,Fall_MA7,R4,R7,Fall_Last7,Fall_100K,Region,Typ
6825,Thüringen,2021-05-05,16,773,2133378,3,388,382,3,20,...,35,1,565.75,643.857143,0.725553,0.95,4507.0,211.26,Thüringen,Bundesland
6826,Thüringen,2021-05-06,16,658,2133378,1,319,335,4,13,...,38,1,653.75,616.714286,1.025490,0.91,4317.0,202.36,Thüringen,Bundesland
6827,Thüringen,2021-05-07,16,587,2133378,0,263,319,5,16,...,29,0,705.25,577.428571,1.354297,0.86,4042.0,189.46,Thüringen,Bundesland
6828,Thüringen,2021-05-08,16,401,2133378,0,192,206,3,17,...,16,0,604.75,558.428571,1.195158,0.85,3909.0,183.23,Thüringen,Bundesland
6829,Thüringen,2021-05-09,16,178,2133378,0,99,78,1,6,...,10,0,456.00,540.142857,0.806010,0.84,3781.0,177.23,Thüringen,Bundesland


In [9]:
df_kreis = df.groupby(by=['Landkreis', 'Meldedatum']).agg({
    'IdLandkreis': 'first',
    'Bundesland': 'first',
    'AnzahlFall': 'sum',
    'Einwohner_x': 'first',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})
    
df_kreis.rename(columns={'Einwohner_x': 'Einwohner'}, inplace=True)


#df_kreis.index.names = ['Region', 'Meldedatum']

df_kreis['Fall_MA4'] = df_kreis.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(4).mean().values
df_kreis['Fall_MA7'] = df_kreis.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(7).mean().values

df_kreis['R4'] = df_kreis['Fall_MA4'].pct_change(periods=4) + 1
df_kreis['R7'] = (df_kreis['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_kreis['Fall_Last7'] = df_kreis['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_kreis['Fall_100K'] = ((df_kreis['Fall_Last7'] / df_kreis['Einwohner']) * 100000).round(2)

df_kreis.reset_index(inplace=True)
df_kreis['Region'] = df_kreis['Landkreis']
df_kreis['Typ'] = 'Landkreis'

df_kreis.tail()

,Landkreis,Meldedatum,IdLandkreis,Bundesland,AnzahlFall,Einwohner,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,...,A80+,unbekannt,Fall_MA4,Fall_MA7,R4,R7,Fall_Last7,Fall_100K,Region,Typ
132930,StadtRegion Aachen,2021-05-05,05334,Nordrhein-Westfalen,157,557026.0,0,83,74,0,...,5,0,102.50,115.428571,0.691400,0.93,808.0,145.06,StadtRegion Aachen,Landkreis
132931,StadtRegion Aachen,2021-05-06,05334,Nordrhein-Westfalen,81,557026.0,0,39,41,1,...,6,0,112.25,105.714286,1.020455,0.82,740.0,132.85,StadtRegion Aachen,Landkreis
132932,StadtRegion Aachen,2021-05-07,05334,Nordrhein-Westfalen,117,557026.0,0,56,61,0,...,3,0,128.50,98.714286,1.498542,0.77,691.0,124.05,StadtRegion Aachen,Landkreis
132933,StadtRegion Aachen,2021-05-08,05334,Nordrhein-Westfalen,53,557026.0,0,26,27,0,...,3,0,102.00,94.428571,1.214286,0.78,661.0,118.67,StadtRegion Aachen,Landkreis
132934,StadtRegion Aachen,2021-05-09,05334,Nordrhein-Westfalen,18,557026.0,0,8,10,0,...,2,0,67.25,91.000000,0.656098,0.79,637.0,114.36,StadtRegion Aachen,Landkreis


In [10]:
## Final dataset
df_merged = pd.concat([df_bund, df_land, df_kreis])

## The geolocations for the districts of Berlin are not included in the dataset, 
## to have Berlin displayed on the choropleth map, I substitute the values for the federal state of Berlin.
df_merged.loc[df_merged['Bundesland'] == 'Berlin', 'IdLandkreis'] = '11000'

df_merged.tail()

,Meldedatum,AnzahlFall,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,...,Einwohner,Fall_100K,Region,Typ,Bundesland,IdBundesland,Fall_MA4,R4,Landkreis,IdLandkreis
132930,2021-05-05,157,0,83,74,0,8,8,52,70,...,557026.0,145.06,StadtRegion Aachen,Landkreis,Nordrhein-Westfalen,NaN,102.50,0.691400,StadtRegion Aachen,05334
132931,2021-05-06,81,0,39,41,1,3,8,18,33,...,557026.0,132.85,StadtRegion Aachen,Landkreis,Nordrhein-Westfalen,NaN,112.25,1.020455,StadtRegion Aachen,05334
132932,2021-05-07,117,0,56,61,0,6,9,50,38,...,557026.0,124.05,StadtRegion Aachen,Landkreis,Nordrhein-Westfalen,NaN,128.50,1.498542,StadtRegion Aachen,05334
132933,2021-05-08,53,0,26,27,0,1,2,23,16,...,557026.0,118.67,StadtRegion Aachen,Landkreis,Nordrhein-Westfalen,NaN,102.00,1.214286,StadtRegion Aachen,05334
132934,2021-05-09,18,0,8,10,0,0,1,5,8,...,557026.0,114.36,StadtRegion Aachen,Landkreis,Nordrhein-Westfalen,NaN,67.25,0.656098,StadtRegion Aachen,05334


In [11]:
if os.path.exists('covid_panel_backup.csv'):
    os.remove('covid_panel_backup.csv')

In [12]:
os.rename('covid_panel.csv', 'covid_panel_backup.csv')

In [13]:
df_merged.to_csv('covid_panel.csv')